In [145]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import ops

In [146]:
df = pd.read_csv('data.csv')
df

,Occupation,Infants,Kids,Adults,Seniors,Normal Need,Clean Need
0,0,0,0,2,0,12,13
1,0,0,2,2,2,37,40
2,0,0,1,2,0,19,16
3,0,0,2,2,1,35,30
4,0,0,1,2,1,23,25
5,0,0,1,2,0,15,13
6,1,1,2,2,1,45,50
7,1,0,2,2,0,35,35
8,1,0,0,2,0,22,20
9,1,1,3,2,0,45,60


In [147]:
X =  np.array(df.iloc[:, :-2]).astype(float)
X.T.shape

(5, 10)

In [148]:
X = X.T
X.shape

(5, 10)

In [149]:
Y_clean = np.array(df.iloc[:, -1]).astype(float)
Y_normal = np.array(df.iloc[:, -2]).astype(float)
Y_clean = Y_clean.T
Y_normal = Y_normal.T

In [150]:
Y_clean = Y_clean.reshape(1, Y_clean.shape[0])
Y_normal = Y_normal.reshape(1, Y_normal.shape[0])
print(Y_clean)
print(Y_normal)

[[13. 40. 16. 30. 25. 13. 50. 35. 20. 60.]]
[[12. 37. 19. 35. 23. 15. 45. 35. 22. 45.]]


In [151]:
W_clean = np.array([[7, 7, 7, 7, 7]]).astype(float)
W_normal = np.array([[15, 15, 8, 5, 12]]).astype(float)
print(W_clean)
print(W_normal)

[[7. 7. 7. 7. 7.]]
[[15. 15.  8.  5. 12.]]


In [152]:
Z_clean = np.dot(W_clean, X)
Z_normal = np.dot(W_normal, X)
print(Z_clean)
print(Y_clean)
print(Z_normal)
print(Y_normal)

[[14. 42. 21. 35. 28. 21. 49. 35. 21. 49.]]
[[13. 40. 16. 30. 25. 13. 50. 35. 20. 60.]]
[[10. 50. 18. 38. 30. 18. 68. 41. 25. 64.]]
[[12. 37. 19. 35. 23. 15. 45. 35. 22. 45.]]


In [153]:
cost_clean = np.sum((Y_clean - Z_clean) ** 2) / Y_clean.shape[1]
print(cost_clean)
cost_normal = np.sum((Y_normal - Z_normal) ** 2) / Y_normal.shape[1]
print(cost_normal)

25.1
117.6


In [154]:
A_clean = tf.Variable(W_clean)
A_normal = tf.Variable(W_normal)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(A_clean))
print(sess.run(A_normal))

[[7. 7. 7. 7. 7.]]
[[15. 15.  8.  5. 12.]]


In [155]:
data = tf.placeholder(dtype = tf.float64, shape = X.shape)
target_clean = tf.placeholder(dtype = tf.float64, shape = Y_clean.shape)
target_normal = tf.placeholder(dtype = tf.float64, shape = Y_normal.shape)

In [156]:
mod_clean = tf.matmul(A_clean, data)
print(mod_clean)
mod_normal = tf.matmul(A_normal, data)
print(mod_normal)

Tensor("MatMul_15:0", shape=(1, 10), dtype=float64)
Tensor("MatMul_16:0", shape=(1, 10), dtype=float64)


In [157]:
loss_clean = tf.reduce_mean(tf.losses.mean_squared_error(labels = target_clean, predictions = mod_clean))
loss_normal = tf.reduce_mean(tf.losses.mean_squared_error(labels = target_normal, predictions = mod_normal))

In [158]:
learning_rate_normal = 0.01
learning_rate_clean = 0.01
iter_num = 5000

In [159]:
optimizer_clean = tf.train.GradientDescentOptimizer(learning_rate_clean).minimize(loss_clean)
optimizer_normal = tf.train.GradientDescentOptimizer(learning_rate_normal).minimize(loss_normal)

In [160]:
# Training
for epoch in range(iter_num):
    sess.run(optimizer_clean, feed_dict = {data: X, target_clean: Y_clean})
    sess.run(optimizer_normal, feed_dict = {data: X, target_normal: Y_normal})
    temp_loss_clean = sess.run(loss_clean, feed_dict = {data: X, target_clean: Y_clean})
    temp_loss_normal = sess.run(loss_normal, feed_dict = {data: X, target_normal: Y_normal})
    if epoch % 500 == 0:
        print("Clean: ",temp_loss_clean)
        print("Normal: ",temp_loss_normal)

Clean:  24.673653
Normal:  97.40403
Clean:  7.9874544
Normal:  4.556997
Clean:  7.8202868
Normal:  3.1204078
Clean:  7.7879777
Normal:  2.8354719
Clean:  7.7784767
Normal:  2.7511184
Clean:  7.775403
Normal:  2.7238324
Clean:  7.774396
Normal:  2.714878
Clean:  7.7740684
Normal:  2.711933
Clean:  7.773955
Normal:  2.710963
Clean:  7.773922
Normal:  2.710645


In [162]:
"""
CLEAN
"""

# Prediction
print(sess.run(mod_clean, feed_dict = {data: X}))

# Original Values
print(Y_clean)

# Starting predictions
print(np.dot(W_clean, X))

[[ 9.31933147 37.85022067 17.32571407 31.59115867 23.58477607 17.32571407
  54.12391518 35.5101677  19.49740249 55.87123579]]
[[13. 40. 16. 30. 25. 13. 50. 35. 20. 60.]]
[[14. 42. 21. 35. 28. 21. 49. 35. 21. 49.]]


In [163]:
"""
NORMAL
"""

# Prediction
print(sess.run(mod_normal, feed_dict = {data: X}))

# Original Values
print(Y_normal)

# Starting predictions
print(np.dot(W_normal, X))

[[10.93951812 38.08838897 17.96430068 31.53873611 24.51395355 17.96430068
  44.76966372 35.5134974  21.46393227 45.24479341]]
[[12. 37. 19. 35. 23. 15. 45. 35. 22. 45.]]
[[10. 50. 18. 38. 30. 18. 68. 41. 25. 64.]]


In [166]:
""" CLEAN """
# Final Weights 
print(sess.run(A_clean))

# Initial Weights
print(W_clean)

[[10.17807103 12.35468548  8.0063826   4.65966573  6.259062  ]]
[[7. 7. 7. 7. 7.]]


In [167]:
""" NORMAL """
# Final Weights 
print(sess.run(A_normal))

# Initial Weights
print(W_normal)

[[10.52441416  2.70651346  7.02478256  5.46975906  6.54965287]]
[[15. 15.  8.  5. 12.]]
